In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Main_Road_Combined_now.csv')

In [3]:
df.head()
#df['dCount']
# df.iloc[0, :]

,Region Name (GO),ONS LACode,ONS LA Name,CP,S Ref E,S Ref N,S Ref Latitude,S Ref Longitude,Road,A-Junction,...,HGVR4,HGVA3,HGVA5,HGVA6,HGV,AMV,max_temp,mean_temp,min_temp,rainfall
0,East Midlands,E10000021,Northamptonshire,6004,458354,270523,52.329706,-1.14512,M1,17,...,10.0,82.0,198,135.0,616.0,3353.0,15.117143,10.667857,6.219286,0.035
1,East Midlands,E10000021,Northamptonshire,6004,458354,270523,52.329706,-1.14512,M1,17,...,22.0,64.0,180,109.0,539.0,3297.0,15.117143,10.667857,6.219286,0.035
2,East Midlands,E10000021,Northamptonshire,6004,458354,270523,52.329706,-1.14512,M1,17,...,25.0,65.0,195,116.0,590.0,2730.0,15.117143,10.667857,6.219286,0.035
3,East Midlands,E10000021,Northamptonshire,6004,458354,270523,52.329706,-1.14512,M1,17,...,15.0,57.0,215,143.0,608.0,2400.0,15.117143,10.667857,6.219286,0.035
4,East Midlands,E10000021,Northamptonshire,6004,458354,270523,52.329706,-1.14512,M1,17,...,20.0,80.0,225,130.0,655.0,2812.0,15.117143,10.667857,6.219286,0.035


In [4]:
# Creating dummy variables
dummies_1 = pd.get_dummies(df['Road'])
step_1 = pd.concat([df, dummies_1], axis=1)

# dummies_2 = pd.get_dummies(df['CP'])
# step_2 = pd.concat([step_1, dummies_2], axis = 1)

dummies_3 = pd.get_dummies(df['RCat'])
step_3 = pd.concat([step_1, dummies_3], axis = 1)

# dummies_4 = pd.get_dummies(df['iDir'])
# step_4 = pd.concat([step_3, dummies_4], axis=1)

In [5]:
#Adding in d, m ,y
step_3['year'] = pd.DatetimeIndex(step_3['dCount']).year
step_3['month'] = pd.DatetimeIndex(step_3['dCount']).month
step_3['day'] = pd.DatetimeIndex(step_3['dCount']).day

In [6]:
df1 = step_3.loc[step_3['Hour'] == 16]

In [7]:
# Dropping variables
df = df1.drop(['Region Name (GO)', 'ONS LACode', 'ONS LA Name', 'S Ref E', 'S Ref N', 'S Ref Latitude', 'S Ref Longitude', 'Road', 'A-Junction', 'A Ref E', 'A Ref N', 'B-Junction', 'B Ref E', 'B Ref N', 'Hour', 'CP', 'RCat', 'iDir', 'BUS', 'LGV', 'HGVR2', 'HGVR3', 'HGVR4','HGVA3','HGVA5', 'HGVA6', 'HGV', 'AMV', 'dCount'], axis =1)

In [8]:
df = df.dropna()

In [9]:
# Creating features/prediction
X = df.drop(['CAR'], axis = 1)
y = df['CAR']

In [10]:
print(X.shape, y.shape)

(145905, 1841) (145905,)


In [11]:
# Train / Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

### SGD Linear Regression

In [12]:
from sklearn.linear_model import SGDRegressor

In [13]:
reg = SGDRegressor()
reg.fit(X_train, y_train)
predictions = reg.predict(X_test)

C:\Apps\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [15]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predictions))

3.6927595249484196e+30


In [16]:
from sklearn .metrics import r2_score
print(r2_score(y_test, predictions))

-4.701114026201469e+24


### Decision tree

In [17]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

In [18]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predictions))

303034.7364223556


In [19]:
from sklearn .metrics import r2_score
print(r2_score(y_test, predictions))

0.6142178118567578


### Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators  = 10, verbose = True)
regr.fit(X_train, y_train)
predictions = regr.predict(X_test)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [21]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predictions))

184221.84677871887


In [22]:
from sklearn .metrics import r2_score
print(r2_score(y_test, predictions))

0.7654740575515082


### Removing weather data

In [23]:
list(X.columns.values)
X1 = X.drop(['max_temp', 'mean_temp', 'min_temp', 'rainfall'], axis = 1)

In [24]:
print(X1.shape, y.shape)

(145905, 1837) (145905,)


In [25]:
# Train / Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.33)

In [26]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

In [27]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predictions))

288717.41044645006


In [28]:
from sklearn .metrics import r2_score
print(r2_score(y_test, predictions))

0.632505708980082


In [29]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators  = 10, verbose = True)
regr.fit(X_train, y_train)
predictions = regr.predict(X_test)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished


In [30]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predictions))

181099.96012521378


In [31]:
from sklearn .metrics import r2_score
print(r2_score(y_test, predictions))

0.7694867055400705


In [55]:
x =  sorted(zip(regr.feature_importances_, list(X1.columns.values)), key=lambda x: x[0], reverse = True)
x

[(0.39301829342587336, 'TM'),
 (0.26109922206937963, '2WMV'),
 (0.04942348880573334, 'PC'),
 (0.040023596152797644, 'day'),
 (0.026532272949752033, 'PA'),
 (0.022650134991252104, 'year'),
 (0.02224365048503616, 'month'),
 (0.021848331375329565, 'Year'),
 (0.006702324828741857, 'M60'),
 (0.004324178435158099, 'M1'),
 (0.003374913763714342, 'A1'),
 (0.003007283388434486, 'M6'),
 (0.002958160591459552, 'M56'),
 (0.0028797474338856837, 'A5103'),
 (0.002700228940529905, 'M25'),
 (0.0026279340619970192, 'A3'),
 (0.002378580822512179, 'A66'),
 (0.002298311939338227, 'M42'),
 (0.0022963774299297425, 'M62'),
 (0.0022544372543970015, 'A2'),
 (0.002206206772953944, 'A19'),
 (0.00214757727767041, 'A34'),
 (0.0020017129807408628, 'A27'),
 (0.001946903328968182, 'A40'),
 (0.0018948645645420874, 'A58'),
 (0.0018281301133498211, 'A38'),
 (0.0018186923041774734, 'M8'),
 (0.001676380772926352, 'A406'),
 (0.0016448729592351643, 'A45'),
 (0.0015534779222818257, 'M20'),
 (0.0013724800743046511, 'M4'),
 (0.

In [56]:
### Hyper parameter tuning
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
### Example
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
rf_random.best_params_